In [ ]:
%load_ext autoreload
%autoreload 2

import env

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# --- 1. Загрузка конфигурации ---
service = env.get_gservice()

if service:
    df_sheet = env.read_df_from_spreadsheet(service, env.SHEET_ID, env.SHEET_NAME)
    print("Данные из Google Sheets загружены")
else:
    raise ConnectionError("Не удалось подключиться к Google API")

RS_TABLE = 'incent_opex_check_universal'
RS_SCHEMA = 'ma_data'

# --- 2. Загрузка алертов за текущий день ---
today = datetime.now().date()

sql_query = f"""
SELECT *
FROM {RS_SCHEMA}.{RS_TABLE}
WHERE date::DATE = '{today}'
  AND is_alert = TRUE
ORDER BY check_name, app, store
"""

df_alerts = env.execute_sql(sql_query)

if df_alerts.empty:
    print(f"Алертов за {today} не найдено.")
else:
    print(f"Загружено {len(df_alerts)} алертов за {today}")


# --- 3. Функция отправки нотификаций для 01-incent.cr ---

def send_cr_notifications(alerts_df, config_row):
    """
    Отправляет Slack-нотификации для алертов 01-incent.cr
    
    Логика:
    - Группировка по app/store
    - Основное сообщение: строки с country=ALL
    - Тред: строки с конкретными странами
    """
    ALERT_NAME = '01-incent.cr'
    ALERT_CATEGORY = config_row['metric_crit_category']
    
    if alerts_df.empty:
        print(f"[{ALERT_NAME}] Алертов нет.")
        return
    
    # Группируем по app/store
    unique_targets = alerts_df[['app', 'store']].drop_duplicates()
    
    arrow_up = ":green_triangle_up_alert:"
    arrow_down = ":red_triangle_down_alert:"
    
    for _, target_row in unique_targets.iterrows():
        t_app = target_row['app']
        t_store = target_row['store']
        
        # Фильтруем данные для текущей группы
        subset = alerts_df[(alerts_df['app'] == t_app) & (alerts_df['store'] == t_store)]
        
        # Заголовок сообщения
        msg_lines = [f"INCENT.OpEx - {ALERT_NAME} ({ALERT_CATEGORY}): *{t_app.upper()} ({t_store})*:"]
        msg_lines_thread = []
        
        # Проходим по строкам группы
        for _, row in subset.iterrows():
            slice1 = row['slice1']  # country
            slice2 = row['slice2']  # level
            slice3 = row['slice3']  # cw
            metric = row['metric']
            current_val = row['current_value']
            change_pct = row['change_perc']
            
            # Определяем иконку
            arrow = arrow_up if change_pct > 0 else arrow_down
            
            # Краткое название метрики
            metric_short = "Prev" if metric == "previous_cr" else "Hist"
            
            # Формируем строку
            change_str = f"{change_pct:+.1%}"
            
            if slice1 == "ALL":
                line = f" {arrow} Lvl {slice2} (cw {slice3}) | CR: {current_val:.2%} | {metric_short} ({change_str})"
                msg_lines.append(line)
            else:
                line_country = f" {arrow} *{slice1}* Lvl {slice2} (cw {slice3}) | CR: {current_val:.2%} | {metric_short} ({change_str})"
                msg_lines_thread.append(line_country)
        
        # Собираем итоговое сообщение
        final_message = "\n".join(msg_lines)
        final_message_thread = "\n".join(msg_lines_thread)
        
        # Отправка в Slack
        try:
            slack = env.SlackNotifier("incent_notifications")
            
            # 1. Отправляем основное сообщение (в канал)
            thread = slack.send_message(final_message)
            
            # 2. Отправляем детали (в тред), ТОЛЬКО если есть что отправлять
            if msg_lines_thread:
                slack.send_message(
                    final_message_thread,
                    thread_ts=thread,
                )
            
            print(f"[{ALERT_NAME}] Отправлено для {t_app}/{t_store}")
        except Exception as e:
            print(f"[{ALERT_NAME}] Ошибка отправки в Slack: {e}")


# --- 4. Функция отправки нотификаций для 02-incent.p7 ---

def send_payers_notifications(alerts_df, config_row):
    """
    Отправляет Slack-нотификации для алертов 02-incent.p7
    
    Логика:
    - Группировка по partner_id (slice1)
    - Основное сообщение: заголовок с названием партнёра
    - Тред: детали по app и сегментам
    """
    ALERT_NAME = '02-incent.p7'
    ALERT_CATEGORY = config_row['metric_crit_category']
    
    if alerts_df.empty:
        print(f"[{ALERT_NAME}] Алертов нет.")
        return
    
    # Индикаторы для разных типов алертов
    ICON_CRITICAL = ":red_circle:"           # Красный для CRITICAL
    ICON_WARNING = ":large_yellow_circle:"   # Жёлтый для WARNING
    
    # Группируем по partner_id (slice1)
    unique_partners = alerts_df['slice1'].unique()
    
    for t_partner in unique_partners:
        subset = alerts_df[alerts_df['slice1'] == t_partner]
        
        # Получаем название партнёра (slice3)
        partner_name = subset['slice3'].iloc[0]
        
        # Определяем общий индикатор: CRITICAL если есть хотя бы один CRITICAL
        has_critical = (subset['alert_category'] == 'CRITICAL').any()
        header_icon = ICON_CRITICAL if has_critical else ICON_WARNING
        
        # Основное сообщение (в канал)
        main_message = f"INCENT.OpEx - {ALERT_NAME} ({ALERT_CATEGORY}): {header_icon} *{partner_name}*"
        
        # Детали по app и сегментам (в тред)
        thread_lines = []
        subset_sorted = subset.sort_values(by=['app', 'alert_category'], ascending=[True, True])
        
        for _, row in subset_sorted.iterrows():
            app = row['app']
            segment = row['slice2']  # operation_segment_nm
            alert_type = row['alert_category']
            payers = int(row['current_value'])
            threshold = int(row['reference_value'])
            
            if alert_type == 'CRITICAL':
                icon = ICON_CRITICAL
                period_info = "4 нед."
            else:
                icon = ICON_WARNING
                period_info = "1 нед."
            
            line = f"{icon} [{alert_type}] {app.upper()} / {segment}: {payers} payers ({period_info}, threshold: {threshold})"
            thread_lines.append(line)
        
        # Отправка в Slack
        try:
            slack = env.SlackNotifier("incent_notifications")
            
            # 1. Отправляем основное сообщение (в канал)
            thread_ts = slack.send_message(main_message)
            
            # 2. Отправляем детали (в тред)
            if thread_lines:
                thread_message = "\n".join(thread_lines)
                slack.send_message(thread_message, thread_ts=thread_ts)
            
            print(f"[{ALERT_NAME}] Отправлено для партнёра {partner_name}")
        except Exception as e:
            print(f"[{ALERT_NAME}] Ошибка отправки в Slack: {e}")


# --- 5. Основной цикл обработки алертов ---

if not df_alerts.empty:
    # Получаем уникальные check_name
    check_names = df_alerts['check_name'].unique()
    
    for check_name in check_names:
        # Получаем конфигурацию проверки из Google Sheets
        try:
            config_row = df_sheet[df_sheet['name'] == check_name].iloc[0]
        except IndexError:
            print(f"[{check_name}] Конфигурация не найдена в Google Sheets. Пропуск.")
            continue
        
        # Проверяем флаг нотификаций
        notification_flag = config_row.get('notification_flag', 'Disabled')
        if notification_flag != 'Enabled':
            print(f"[{check_name}] Нотификации отключены (notification_flag != Enabled).")
            continue
        
        # Фильтруем алерты для текущей проверки
        check_alerts = df_alerts[df_alerts['check_name'] == check_name].copy()
        
        print(f"\n[{check_name}] Обработка {len(check_alerts)} алертов...")
        
        # Вызываем соответствующую функцию отправки
        if check_name == '01-incent.cr':
            send_cr_notifications(check_alerts, config_row)
        elif check_name == '02-incent.p7':
            send_payers_notifications(check_alerts, config_row)
        else:
            print(f"[{check_name}] Обработчик нотификаций не определён.")

print(f"\nЗавершено: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")